In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
from scipy.stats import mannwhitneyu
import seaborn as sns
from itertools import combinations



In [ ]:

df_save_path = 'tmp/evaluation_results'

results_df = pd.read_pickle(df_save_path)
algos = results_df['algo'].unique().tolist()


In [ ]:

for algo in results_df['algo'].unique():

    trained_rewards = results_df[results_df['algo']==algo][results_df['trained_model']==True]['rewards'].tolist()
    untrained_rewards = results_df[results_df['algo']==algo][results_df['trained_model']==False]['rewards'].tolist()
    print(algo)
    sns.displot(results_df[results_df['algo']==algo], x="rewards", hue="trained_model", kind="kde", fill=True, label=algo)
    print(mannwhitneyu(trained_rewards,untrained_rewards))
    print(mannwhitneyu(untrained_rewards,trained_rewards))

In [ ]:

algo_combinations = list(combinations(algos,2))
algo_combinations

In [ ]:
for algo_combination in algo_combinations:
    print(algo_combination)
    trained_rewards_algo = [results_df[results_df['algo']==algo][results_df['trained_model']==True]['rewards'].tolist() for algo in algo_combination]

    sns.displot(results_df[(results_df['trained_model']==True) & (results_df['algo'].isin(algo_combination))], x="rewards", hue="algo", kind="kde", fill=True)
    print(mannwhitneyu(trained_rewards_algo[0],trained_rewards_algo[1]))
    print(mannwhitneyu(trained_rewards_algo[1],trained_rewards_algo[0]))

In [ ]:
for algo_combination in algo_combinations:
    print(algo_combination)
    trained_rewards_algo = [results_df[results_df['algo']==algo][results_df['trained_model']==True]['rewards'].tolist() for algo in algo_combination]

    sns.displot(results_df[(results_df['trained_model']==True) & (results_df['algo'].isin(algo_combination))], x="rewards", hue="algo", kind="kde", fill=True)
    print(mannwhitneyu(trained_rewards_algo[0],trained_rewards_algo[1]))
